In [10]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import time
from binance.client import Client


# Your Binance API credentials
api_key = 'Ta0xEGr3ogYlT7L4hxzgMV8zIOxDGChCqbdHuV1ZxG6pqCtdvSJ1KS5ZMq2XX9Uk'
api_secret = 'JqwryeIAHF1aISI0G1Z4X8zIRwU9YIoUIWCq3OOC3OpZrs1BMi1i7iv369aDZTp4'


# Initialize the Binance client
client = Client(api_key, api_secret)

# Function to fetch historical data using python-binance
def get_binance_data(symbol, interval, start_time, end_time):
    # Fetch historical klines (candlestick data)
    klines = client.get_historical_klines(symbol, interval, start_time, end_time)

    # Convert the data into a pandas DataFrame
    df = pd.DataFrame(klines, columns=[
        'timestamp', 'Open', 'High', 'Low', 'Close', 'Volume', 'close_time',
        'quote_asset_volume', 'num_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'
    ])
    
    # Convert timestamp to a readable format
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

    return df

# Define the start and end time for the data (Binance expects a string for dates)
start_time = "2024-09-22 00:00:00"
end_time = datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S")

# Fetch the data
binance_df = get_binance_data("BTCUSDT", Client.KLINE_INTERVAL_1MINUTE, start_time, end_time)

# Save to CSV
binance_df.to_csv("binance_btcusdt_full.csv", index=False)
print("Full Binance data saved.")

Full Binance data saved.


In [22]:
import requests
import pandas as pd
from datetime import datetime, timedelta

# Function to fetch candles from Gate.io
def fetch_gateio_candles(symbol, interval, limit=1000, start_time=None):
    url = f"https://api.gateio.ws/api/v4/spot/candlesticks"
    params = {
        'currency_pair': symbol,
        'interval': interval,
        'limit': limit  # Maximum limit per request
    }

    if start_time:
        params['from'] = int(start_time.timestamp())  # Gate.io expects UNIX time in seconds

    response = requests.get(url, params=params)

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching data from Gate.io: {response.status_code}, Response: {response.text}")
        return []

# Function to fetch last 6 days of 1-minute candles
def get_gateio_last_6_days(symbol, interval):
    all_data = []
    current_time = datetime.utcnow()
    start_time = current_time - timedelta(days=7)  # Start from 6 days ago

    print(f"Fetching data from {start_time} to {current_time}...")
    candles = fetch_gateio_candles(symbol, interval, start_time=start_time)

    if candles:
        all_data.extend(candles)
    else:
        print("No data received or an error occurred.")

    # Inspect the first entry to see the structure of the returned data
    if all_data:
        print(f"Sample data: {all_data[0]}")  # Print the first row to inspect the data

    # Create the DataFrame with the correct number of columns based on the data returned
    columns = ['timestamp', 'Close', 'Volume', 'High', 'Low', 'Open', 'quote_volume', 'extra_column']  # Adjust the columns as needed
    df = pd.DataFrame(all_data, columns=columns)

    # Convert the timestamp to a readable datetime
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s', utc=True)

    return df

# Example usage for Gate.io:
gateio_df = get_gateio_last_6_days("BTC_USDT", "1m")  # Fetch last 6 days of 1-minute candles from Gate.io

# Save the data to a CSV file
if not gateio_df.empty:
    gateio_df.to_csv("gateio_btcusdt_last_6_days.csv", index=False)
    print("Full Gate.io data saved.")
else:
    print("No data to save.")

Fetching data from 2024-09-21 07:24:07.961317 to 2024-09-28 07:24:07.961317...
Error fetching data from Gate.io: 400, Response: {"label":"INVALID_PARAM_VALUE","message":"Candlestick too long ago. Maximum 10000 points ago are allowed"}
No data received or an error occurred.
No data to save.
